In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

print(tf.__version__)

2022-10-09 14:50:06.261873: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-09 14:50:06.412698: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-09 14:50:06.870816: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-10-09 14:50:06.870881: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

2.10.0


In [2]:
data = np.loadtxt('foo.csv',delimiter=',')
print(data[0:5,:])

[[  3.20449854   2.85800218  -4.33533806  -0.97209247  -8.07829853
   11.93423825  -4.3051714    1.76192875  -4.33438682   1.77388542]
 [  3.06064764   2.34991961  -3.50841735  -4.63072136 -12.0463229
   21.08299674  -2.07948816   4.44471054  -2.06438134   4.46291992]
 [  2.74348635   1.6765937   -2.25636552  -8.04949242 -14.62594151
   28.62500266  -2.06447959   4.46280148  -2.06438134   4.46291992]
 [  2.25738003   0.91536564  -0.68738097 -11.36504141 -15.5298887
   33.6880841   -2.06438216   4.46291893  -2.06438134   4.46291992]
 [  1.61144012   0.15330298   1.06578918 -14.41187267 -14.65920691
   35.96090489  -2.06438134   4.46291991  -2.06438134   4.46291992]]


In [3]:
rows,cols = data.shape
dt = 0.05

x_data = data[0:-1,0:8]
u_data=data[0:-1,-2:]
x_dot_data = (data[1:,3:8]-data[0:-1,3:8])/dt


In [4]:
tf.compat.v1.disable_eager_execution()
X = tf.compat.v1.placeholder(shape=(8,),dtype='float32')
U = tf.compat.v1.placeholder(shape=(2,),dtype='float32')
X_dot = tf.compat.v1.placeholder(shape=(5,),dtype='float32')
c = tf.Variable(np.random.random((4,)),trainable=True,name='c',dtype='float32')
d = tf.Variable(np.random.random((3,)),trainable=True,name='d',dtype='float32')
theta = tf.Variable(np.random.random()*1e-4,trainable=True,dtype='float32')
tau = tf.Variable(np.random.random()*1e-2,trainable=True,dtype='float32')

In [5]:
learning_rate = 0.01
training_epochs=5000

In [6]:
X_dot_pred = tf.stack([-(c[0]*1e3)/(theta*1e4)*(X[0]-X[-2])-(c[1]*1e3)/(theta*1e4)*(X[0]-X[1])-(d[0]*1e5)/(theta*1e4)*X[3],
           -(c[1]*1e3)/(theta*1e4)*(X[1]-X[0])-(c[2]*1e3)/(theta*1e4)*(X[1]-X[2])-(d[1]*1e5)/(theta*1e4)*X[4],
           -(c[2]*1e3)/(theta*1e4)*(X[2]-X[1])-(c[3]*1e3)/(theta*1e4)*(X[2]-X[-1])-(d[2]*1e5)/(theta*1e4)*X[5],
           1/(tau*1e2)*(U[0] - X[-2]),
           1/(tau*1e2)*(U[1] - X[-1])])

In [12]:
X_dot_pred = tf.stack([-(c[0]*1e3)/(theta*1e4)*(X[:,0]-X[:,-2])-(c[1]*1e3)(theta*1e4)*(X[:,0]-X[:,1])-(d[0]*1e5)/(theta*1e4)*X[:,3],
            -(c[1]*1e3)/(theta*1e4)*(X[:,1]-X[:,0])-(c[2]*1e3)/(theta*1e4)*(X[:,1]-X[:,2])-(d[1]*1e5)/(theta*1e4)*X[:,4],
            -(c[2]*1e3)/(theta*1e4)*(X[:,2]-X[:,1])-(c[3]*1e3)/(theta*1e4)*(X[:,2]-X[:,-1])-(d[2]*1e5)/(theta*1e4)*X[:,5],
            1/(tau*1e2)*(U[:,0] - X[:,-2]),
            1/(tau*1e2)*(U[:,1] - X[:,-1])])

ValueError: Index out of range using input dim 1; input has only 1 dims for '{{node strided_slice_29}} = StridedSlice[Index=DT_INT32, T=DT_FLOAT, begin_mask=1, ellipsis_mask=0, end_mask=1, new_axis_mask=0, shrink_axis_mask=2](Placeholder, strided_slice_29/stack, strided_slice_29/stack_1, strided_slice_29/stack_2)' with input shapes: [8], [2], [2], [2] and with computed input tensors: input[3] = <1 1>.

In [8]:
X_dot_pred.shape

TensorShape([5])

In [9]:
cost = tf.reduce_sum(tf.pow(X_dot_pred-X_dot, 2)) / (3 * rows)
optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate).minimize(cost)
init = tf.compat.v1.global_variables_initializer()

In [10]:
with tf.compat.v1.Session() as sess:
    sess.run(init)
    for epoch in range(training_epochs):
        #sess.run(optimizer, feed_dict = {X : x_data, U : u_data,X_dot:x_dot_data})
        for (_x, _u,_x_dot) in zip(x_data, u_data,x_dot_data):
            sess.run(optimizer, feed_dict = {X : _x, U : _u,X_dot:_x_dot})
        co = sess.run(cost, feed_dict = {X : _x, U : _u,X_dot:_x_dot})
        print("Epoch", (epoch + 1), ": cost =", co, "c =", sess.run(c), "d =", sess.run(d), "theta =", sess.run(theta), "tau =", sess.run(tau))

    #training_cost = sess.run(cost, feed_dict = {X : _x, U : _u,X_dot:_x_dot})


2022-10-09 14:50:08.109729: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-09 14:50:08.171373: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-09 14:50:08.171540: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-09 14:50:08.172098: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Epoch 1 : cost = 0.108094156 c = [ 44839.64     1021.0829 565446.1    460839.03  ] d = [  -580424.7  -2444220.  -90200368. ] theta = 5392405400000.0 tau = 0.0021114366
Epoch 2 : cost = 0.108094156 c = [ 44839.64     1021.0829 565446.1    460839.03  ] d = [  -580424.7  -2444220.  -90200368. ] theta = 5392405400000.0 tau = 0.17783669
Epoch 3 : cost = 0.108094156 c = [ 44839.64     1021.0829 565446.1    460839.03  ] d = [  -580424.7  -2444220.  -90200368. ] theta = 5392405400000.0 tau = 0.17783529
Epoch 4 : cost = 0.108094156 c = [ 44839.64     1021.0829 565446.1    460839.03  ] d = [  -580424.7  -2444220.  -90200368. ] theta = 5392405400000.0 tau = 0.17783388
Epoch 5 : cost = 0.108094156 c = [ 44839.64     1021.0829 565446.1    460839.03  ] d = [  -580424.7  -2444220.  -90200368. ] theta = 5392405400000.0 tau = 0.17783248
Epoch 6 : cost = 0.108094156 c = [ 44839.64     1021.0829 565446.1    460839.03  ] d = [  -580424.7  -2444220.  -90200368. ] theta = 5392405400000.0 tau = 0.17783108
Ep

KeyboardInterrupt: 